In [1]:
!pip3 install --user ipykernel
!python3 -m ipykernel install --user --name=aue8930
!jupyter kernelspec list
#!jupyter kernelspec uninstall aue8930

Installed kernelspec aue8930 in /home/csamak/.local/share/jupyter/kernels/aue8930
Available kernels:
  aue8930    /home/csamak/.local/share/jupyter/kernels/aue8930
  python3    /home/csamak/.local/share/jupyter/kernels/python3


In [2]:
import models
import os
import torch
from torch import optim, nn
from torchvision import transforms, datasets
import torchvision
import common

In [3]:
#DIRECTORY SETTINGS
os.chdir("..")#Go up two directories
SAVE_DIR = 'models'
MODEL_SAVE_PATH = os.path.join(SAVE_DIR, 'batch_size.pt')


#HYPERPARAMETERS
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

EPOCHS=100
BATCH_SIZE = 256
criterion = nn.CrossEntropyLoss()
ADAM_OPTIMISER=True
LEARNING_RATE=0.001

In [4]:
train_transforms = transforms.Compose([# Data Transforms
                           transforms.Resize(256),#Resize
                           transforms.RandomHorizontalFlip(30),#Flip
                           transforms.RandomRotation(10),#Roatate
                           transforms.RandomCrop(256),#Crop
                           transforms.ToTensor(),#Convert to Tensor
                           transforms.Normalize((0.49139968, 0.48215827, 0.44653124), (0.24703233, 0.24348505, 0.26158768))#Normalize
                       ])

test_transforms = transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.49139968, 0.48215827, 0.44653124), (0.24703233, 0.24348505, 0.26158768))
                       ])

In [5]:
train_data = torchvision.datasets.CIFAR10(root='data', train=True, download=True, transform=train_transforms)#Use CIFAR10 to train
train_data, valid_data = torch.utils.data.random_split(train_data, [int(len(train_data)*0.9), len(train_data) - int(len(train_data)*0.9)])
test_data = torchvision.datasets.CIFAR10(root='data', train=False, download=True, transform=test_transforms)


print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')


train_iterator = torch.utils.data.DataLoader(train_data, shuffle=True, batch_size=BATCH_SIZE)
valid_iterator = torch.utils.data.DataLoader(valid_data, batch_size=BATCH_SIZE)
test_iterator = torch.utils.data.DataLoader(test_data, batch_size=BATCH_SIZE)

Files already downloaded and verified
Files already downloaded and verified
Number of training examples: 45000
Number of validation examples: 5000
Number of testing examples: 10000


In [6]:
model = torchvision.models.resnet18(pretrained=True)#TorchVision

for param in model.parameters():
    param.requires_grad = False
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10)
model = model.to(device)

#Hyperparameters
if(ADAM_OPTIMISER):
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
else:
    optimizer = optim.SGD(model.classifier.parameters(), lr=0.001, momentum=0.5)

In [7]:
#Train
best_valid_loss = float('inf')
for epoch in range(EPOCHS):#Range of Epochs
    print(epoch)
    train_loss, train_acc = common.train(model, device, train_iterator, optimizer, criterion)#Train Loss Calculation
    valid_loss, valid_acc = common.evaluate(model, device, valid_iterator, criterion)#Validation Loss Calculation

    if valid_loss < best_valid_loss:#Validation Loss - Is current lower than the saved validation loss.
        best_valid_loss = valid_loss#Save the best loss (lowest)
        torch.save(model.state_dict(), MODEL_SAVE_PATH)#Save the model

    print(f'| Epoch: {epoch+1:02} | Train Loss: {train_loss:.3f} | Train Acc: {train_acc*100:05.2f}% | Val. Loss: {valid_loss:.3f} | Val. Acc: {valid_acc*100:05.2f}% |')



0
| Epoch: 01 | Train Loss: 1.275 | Train Acc: 60.73% | Val. Loss: 0.926 | Val. Acc: 70.97% |
1
| Epoch: 02 | Train Loss: 0.853 | Train Acc: 72.32% | Val. Loss: 0.796 | Val. Acc: 73.76% |
2
| Epoch: 03 | Train Loss: 0.773 | Train Acc: 74.21% | Val. Loss: 0.767 | Val. Acc: 73.71% |
3
| Epoch: 04 | Train Loss: 0.736 | Train Acc: 74.97% | Val. Loss: 0.734 | Val. Acc: 75.10% |
4
| Epoch: 05 | Train Loss: 0.716 | Train Acc: 75.69% | Val. Loss: 0.702 | Val. Acc: 75.52% |
5
| Epoch: 06 | Train Loss: 0.700 | Train Acc: 75.96% | Val. Loss: 0.683 | Val. Acc: 76.79% |
6
| Epoch: 07 | Train Loss: 0.690 | Train Acc: 76.47% | Val. Loss: 0.689 | Val. Acc: 77.23% |
7
| Epoch: 08 | Train Loss: 0.677 | Train Acc: 76.71% | Val. Loss: 0.676 | Val. Acc: 76.25% |
8
| Epoch: 09 | Train Loss: 0.670 | Train Acc: 76.77% | Val. Loss: 0.668 | Val. Acc: 77.11% |
9
| Epoch: 10 | Train Loss: 0.665 | Train Acc: 77.01% | Val. Loss: 0.673 | Val. Acc: 77.20% |
10
| Epoch: 11 | Train Loss: 0.659 | Train Acc: 77.31% | Val

In [8]:
#3. OUTPUT

model.load_state_dict(torch.load(MODEL_SAVE_PATH)) #Load best weights from file
test_loss, test_acc = common.evaluate(model, device, valid_iterator, criterion) #Test Loss is dependent on
print(f'| Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:05.2f}%')

| Test Loss: 0.624 | Test Acc: 78.39%
